In [3]:
import os
import pandas as pd
import numpy as np
import plotnine
from datetime import date
import scipy
import seaborn as sns
from Bio import SeqIO
import matplotlib.pyplot as plt
from upsetplot import from_memberships

In [4]:
## GLOBAL PARAMETERS ##
base_dir = "/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_trials_September21"
eukulele_dir_designer = "EUKulele_designer"
eukulele_dir_CAG = "EUKulele_07-CAG"
eukulele_dir_MAD = "EUKulele_12-MAD"
salmon_dir = os.path.join(base_dir,"08-salmon_mapping")

In [5]:
## FUNCTIONS ##

def read_eggnog_annots(chosen_dir, chosen_community):
    eggnog_all = pd.DataFrame()
    ### SMALL-SCALE EXAMPLE - community 1, trial 1, 100k ####
    eggnog_designer = pd.read_csv(os.path.join(base_dir, chosen_dir, "eggnog_designer", 
                       "designer_assembly_" + str(chosen_community) + ".emapper.annotations"),
                                  sep="\t",comment="#",
                            header=None,names=["query","seed_ortholog","evalue","score",
                                               "eggNOG_OGs","max_annot_lvl","COG_category",
                                               "Description","Preferred_name","GOs","EC","KEGG_ko",
                                               "KEGG_Pathway","KEGG_Module","KEGG_Reaction","KEGG_rclass",
                                               "BRITE","KEGG_TC","CAZy","BiGG_Reaction","PFAMs"])
    eggnog_designer["Source"] = "Designer"
    eggnog_all = pd.concat([eggnog_all,eggnog_designer])
    eggnog_CAG = pd.read_csv(os.path.join(base_dir, chosen_dir, "eggnog_CAG", 
                       "sim_raw_reads_" + str(chosen_community) + ".emapper.annotations"),
                             sep="\t",comment="#",
                            header=None,names=["query","seed_ortholog","evalue","score",
                                               "eggNOG_OGs","max_annot_lvl","COG_category",
                                               "Description","Preferred_name","GOs","EC","KEGG_ko",
                                               "KEGG_Pathway","KEGG_Module","KEGG_Reaction","KEGG_rclass",
                                               "BRITE","KEGG_TC","CAZy","BiGG_Reaction","PFAMs"])
    eggnog_CAG["Source"] = "CAG"
    eggnog_all = pd.concat([eggnog_all,eggnog_CAG])
    assemblers = ["spades","rnaspades","trinity","megahit"]
    eggnog_all_assemblers = pd.DataFrame()
    for assembler in assemblers:
        eggnog_assembler = pd.read_csv(os.path.join(base_dir, chosen_dir, "eggnog_assembler", 
                       "sim_raw_reads_" + str(chosen_community) + "_" + str(assembler) + ".emapper.annotations"),
                                       sep="\t",comment="#",
                            header=None,names=["query","seed_ortholog","evalue","score",
                                               "eggNOG_OGs","max_annot_lvl","COG_category",
                                               "Description","Preferred_name","GOs","EC","KEGG_ko",
                                               "KEGG_Pathway","KEGG_Module","KEGG_Reaction","KEGG_rclass",
                                               "BRITE","KEGG_TC","CAZy","BiGG_Reaction","PFAMs"])
        
        eggnog_assembler["Source"] = assembler
        eggnog_all = pd.concat([eggnog_all,eggnog_assembler])
    
    eggnog_all["jEUKebox"] = chosen_dir
    if "100k" in chosen_dir:
        eggnog_all["MMETSPGroup"] = "A"
    else:
        eggnog_all["MMETSPGroup"] = "B"
        
    return eggnog_all

def compute_sourmash_index(sourmash_file, community_file):
    sourmash_file_T = sourmash_file.copy()
    sourmash_file_T.columns = [curr.split("/")[-1] for curr in sourmash_file.columns]
    sourmash_file_T["Protein_Files_Split"] = sourmash_file_T.columns
    community_file["Protein_Files_Split"] = [curr.split("/")[-1] for curr in community_file.Protein_Files] 
    community_file = community_file.groupby(["Community","Protein_Files_Split"])["Proportion"].sum().reset_index()
    trad_diversity = community_file.groupby("Community")["Proportion"].\
                        agg([lambda x: -np.sum(x * np.log(x)), len]).reset_index()
    trad_diversity.columns = ["Community","Shannon","Richness"]
    
    cols_select = ["Community","Proportion","Protein_Files_Split"]
    cols_select.extend([curr.split("/")[-1] for curr in sourmash_file.columns])
    merged_file = community_file.merge(sourmash_file_T,
                                       left_on="Protein_Files_Split",
                                       right_on="Protein_Files_Split",
                                       how="left")[cols_select]
    
    wide_to_long_merged = pd.wide_to_long(merged_file,stubnames="MMETSP",
                i=["Community","Proportion","Protein_Files_Split"],sep="",
                j="MMETSP_id",suffix='\w+_clean.pep.fa').reset_index() 
    wide_to_long_merged.MMETSP_id = ["MMETSP" + curr for curr in wide_to_long_merged.MMETSP_id]
    
    community_file_join = community_file.copy()[["Protein_Files_Split","Proportion"]]
    community_file_join.columns = ["Joined_ID","Joined_ID_prop"]

    combined_merged = wide_to_long_merged.merge(community_file_join,
                              right_on="Joined_ID",
                              left_on="MMETSP_id",how="right")
    
    combined_merged["Score"] = [(1-sourmash) * min(prop1,prop2) for sourmash,prop1,prop2 in \
                        zip(combined_merged.MMETSP, combined_merged.Proportion,\
                        combined_merged.Joined_ID_prop)]
    
    final_scores = combined_merged.groupby("Community").sum().reset_index().merge(trad_diversity,how="inner")
    
    return final_scores

# Creates a dataframe that is the processed result for this community of clustering
# the designer proteins with the reassembled products from eukrhythmic
def cluster_sim_designer(cluster_file):
    clust_designer = pd.read_csv(cluster_file, sep = "\t",
                             header = None, names = ["Representative","Member"])
    
    clust_designer["RepresentativeType"] = ["designer" if "CAMPEP" in curr else "reassembled" if "sim_raw_reads" in\
                                            curr else "indeterminate" for curr in clust_designer.Representative]
    clust_designer["MemberType"] = ["designer" if "CAMPEP" in curr else "reassembled" if "sim_raw_reads" in\
                                            curr else "indeterminate" for curr in clust_designer.Member]
    
    def to_set(x):
        return set(x)

    def length_set(x):
        return len(set(x))

    clust_designer_sets = clust_designer.groupby("Representative").agg({"MemberType": to_set}).reset_index()

    conditions = [
        (clust_designer_sets["MemberType"] == set({"designer","reassembled"})),
        (clust_designer_sets["MemberType"] == set({"designer"})),
        (clust_designer_sets["MemberType"] == set({"reassembled"})),
    ]
    choices = ["both","designer","reassembled"]

    clust_designer_sets["GroupType"] = np.select(conditions, choices)
    combined_result = clust_designer_sets.merge(clust_designer, 
                                                left_on="Representative",
                                                right_on="Representative", how="outer")

    combined_result["id_shortened"] = [curr.split(".p1")[0] + ".p1" if ".p1" in curr \
                                         else curr.split(".p2")[0] + ".p2" if ".p2" in curr \
                                         else curr.split(".p3")[0] + ".p3" if ".p3" in curr \
                                         else curr.split(".p4")[0] + ".p4" if ".p4" in curr \
                                         else curr.split(".p5")[0] + ".p5" if ".p5" in curr \
                                         else curr.split(".p6")[0] + ".p6" if ".p6" in curr \
                                         else curr.split(".p7")[0] + ".p7" if ".p7" in curr \
                                         else curr for curr in combined_result.Member]
    return combined_result

# convert assembler clusters to well-defined names
def convert_names(eggnog_file, cluster_result):
    if eggnog_file is None:
        return eggnog_file
    eggnog_file["no_ext"] = [curr.split(".")[0] for curr in eggnog_file["query"]]

    def to_set(x):
        return set(x)

    def length_set(x):
        return len(set(x))

    eggnog_clust_informed = cluster_result.merge(eggnog_file, left_on = ["Member","Community","Name"],
                                                 right_on = ["no_ext","Community","Name"],
                                                 how = "left")\
                           .groupby(["Representative","Community","Name"])\
                           .agg({"Assembler_Mem": to_set}).reset_index()

    conditions = [
        (eggnog_clust_informed["Assembler_Mem"] == set({"megahit","trinity","spades","rnaspades"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"megahit","trinity"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"spades","rnaspades"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"trinity","rnaspades"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"rnaspades"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"trinity"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"megahit"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"spades"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"spades","rnaspades","trinity"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"spades","rnaspades","megahit"})),
        (eggnog_clust_informed["Assembler_Mem"] == set({"rnaspades","megahit"})),
        ([len(curr) == 3 for curr in eggnog_clust_informed["Assembler_Mem"]]),
        ([len(curr) == 2 for curr in eggnog_clust_informed["Assembler_Mem"]]),
    ]
    choices = ["all","MT","SR","TR","R","T","M","S","SRT","SRM","RM","other3","other2"]

    eggnog_clust_informed["GroupType"] = np.select(conditions, choices)
    
    return eggnog_clust_informed

# Read in annotations from eggNOG-mapper.
def read_eggnog_results(base_dir, jeukebox_dir, eggnog_dir, eggnog_prefix, sim_file,
                        eggnog_names, read_salmon = True, 
                        salmon_stub = "sim_raw_reads_", 
                        is_prot = False,
                        prot_dict = None,
                        salmon_dir = os.path.join("eukrhythmic_assembly",
                                                                                     "intermediate-files",
                                                                                     "04-compare",
                                                                                     "09-CAG-mapping",
                                                                                     "salmon")):
    number_file = sim_file.split("_merged")[0].split("_")[-1]
    if read_salmon:
        if salmon_dir is not None:
            if "09-CAG-mapping" in salmon_dir:
                salmon_dir = os.path.join(base_dir, jeukebox_dir, salmon_dir)
                
            
            if os.path.isfile(os.path.join(salmon_dir,str(number_file) + "_quant",
                                 "quant.sf")):
                salmon_file = pd.read_csv(os.path.join(salmon_dir,str(number_file) + "_quant",
                                     "quant.sf"), sep = "\t")
                salmon_file["NewName"] = [str(curr).split("_flag")[0].split("_len=")[0] \
                                          for curr in salmon_file["Name"]]
            elif not os.path.isfile(os.path.join(salmon_dir,salmon_stub + str(number_file) + "_quant",
                                 "quant.sf")):
                read_salmon = False
            else:
                salmon_file = pd.read_csv(os.path.join(salmon_dir,salmon_stub + str(number_file) + "_quant",
                                     "quant.sf"), sep = "\t")
                salmon_file["NewName"] = [str(curr).split("_flag")[0].split("_len=")[0] \
                                          for curr in salmon_file["Name"]]
                #salmon_file["NewName"] = ["_sim_raw_reads".join(curr.split("_sim_raw_reads")[0:2]) for curr \
                #                          in salmon_file["NewName"]]
            
    em_file_path = os.path.join(base_dir, jeukebox_dir,
                               eggnog_dir, eggnog_prefix + \
                               str(sim_file.split("_merged")[0].split("_")[-1]) +
                               ".emapper.annotations")
    if os.path.isfile(em_file_path):
        eggnog_curr = pd.read_csv(em_file_path, 
                                  sep = "\t", comment = "#", header = None, names = eggnog_names)
        
        eggnog_curr["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
        eggnog_curr["Name"] = jeukebox_dir
        if read_salmon:
            if is_prot:
                salmon_file = salmon_file.rename(columns={"Name":"nucl_query"})
                eggnog_curr["nucl_query"] = [prot_dict[curr] for curr in eggnog_curr["query"]]
                eggnog_curr = eggnog_curr.merge(salmon_file, left_on = "nucl_query", right_on = "nucl_query")
            else:
                salmon_file = salmon_file.rename(columns={"Name":"query"})
                eggnog_curr["query"] = [curr.split(".")[0] for curr in eggnog_curr["query"]]
                eggnog_curr = eggnog_curr.merge(salmon_file, left_on = "query", right_on = "query")
        return eggnog_curr
    return None

# Read in EUKulele annotations with salmon quantification.
def read_EUKulele_annotations(base_dir, jeukebox_dir, eukulele_dir, sim_file,
                              filename_stub = "sim_raw_reads_", suffix = "_merged", salmon_stub = "sim_raw_reads_",
                              salmon_dir = os.path.join("eukrhythmic_assembly","intermediate-files",
                      "04-compare","09-CAG-mapping","salmon")):
    if salmon_dir is not None:
        if "09-CAG-mapping" in salmon_dir:
            salmon_dir = os.path.join(base_dir, jeukebox_dir, salmon_dir)
    if os.path.isdir(os.path.join(base_dir, jeukebox_dir, eukulele_dir)):
        if salmon_dir is not None:
            if not os.path.isdir(salmon_dir):
                return pd.DataFrame()
        number_file = sim_file.split("_merged")[0].split("_")[-1]
        if not os.path.isfile(os.path.join(base_dir, jeukebox_dir, eukulele_dir,
                                                "taxonomy_estimation", filename_stub + \
                                                 str(sim_file.split("_merged")[0].split("_")[-1]) +\
                                                suffix + "-estimated-taxonomy.out")):
            return pd.DataFrame()
        EUKulele_CAG = pd.read_csv(os.path.join(base_dir, jeukebox_dir, eukulele_dir,
                                                "taxonomy_estimation", filename_stub + \
                                                 str(sim_file.split("_merged")[0].split("_")[-1]) +\
                                                suffix + "-estimated-taxonomy.out"), sep = "\t")
        if salmon_dir is not None:
            salmon_file = pd.read_csv(os.path.join(salmon_dir,salmon_stub + str(number_file) + "_quant",
                                 "quant.sf"), sep = "\t")
            salmon_file["NewName"] = [str(curr).split("_flag")[0].split("_len=")[0] \
                                      for curr in salmon_file["Name"]]
            salmon_file["NewName"] = ["_sim_raw_reads".join(curr.split("_sim_raw_reads")[0:2]) for curr \
                                      in salmon_file["NewName"]]
            EUKulele_CAG = pd.merge(left = salmon_file, right = EUKulele_CAG, how = "outer",
                                    left_on = "Name", right_on = "transcript_name")
            EUKulele_CAG["PivotName"] = EUKulele_CAG["Name"]
        else:
            EUKulele_CAG["PivotName"] = EUKulele_CAG["transcript_name"]

        EUKulele_CAG["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
        EUKulele_CAG["Name"] = jeukebox_dir
        EUKulele_CAG["Assembler"] = [str(curr).split("_")[0] if curr == curr else \
                                     "None" for curr in EUKulele_CAG.PivotName]

        EUKulele_CAG[["Domain","Supergroup","Phylum","Class","Order",
                    "Family","Genus","Species"]]= EUKulele_CAG["full_classification"].\
                                            str.split(";", n = 8, expand = True)
        EUKulele_CAG[["Domain","Supergroup","Phylum","Class",\
                  "Family","Genus","Species","classification"]] = EUKulele_CAG[["Domain","Supergroup","Phylum","Class",\
                  "Family","Genus","Species","classification"]].fillna(value="Unannotated")
        
        EUKulele_CAG["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
        EUKulele_CAG["Name"] = jeukebox_dir
        return EUKulele_CAG
    
def format_sim_des(clust_designer):
    clust_designer["RepresentativeType"] = ["designer" if "CAMPEP" in curr else "reassembled" if "sim_raw_reads" in\
                                        curr else "indeterminate" for curr in clust_designer.Representative]
    clust_designer["MemberType"] = ["designer" if "CAMPEP" in curr else "reassembled" if "sim_raw_reads" in\
                                        curr else "indeterminate" for curr in clust_designer.Member]
    
    return clust_designer

# compare identified GO terms to the designer assembly.
def compare_to_designer_go(eggnog_designer_curr, final_res_GOs):
    eggnog_designer_curr["GOs"] = [curr.split(",") for curr in eggnog_designer_curr.GOs]
    eggnog_designer_curr = eggnog_designer_curr.explode("GOs")
    eggnog_designer_GOs = eggnog_designer_curr.groupby(["GOs"])["query"].count().reset_index()
    eggnog_designer_GOs = eggnog_designer_GOs.rename({"query":"DesignerCount"},axis="columns")
    
    final_res_GOs = final_res_GOs.groupby(["Community","Name","GOs",
                                           "AssemblerCluster","SimDesignerCluster"])["query"].count().\
        reset_index()
    final_res_GOs = final_res_GOs.rename({"query":"CAGCount"},axis="columns")
    
    to_return = eggnog_designer_GOs.merge(final_res_GOs, left_on = ["GOs"],
                                     right_on = ["GOs"],
                              how = "outer").fillna(0)
    
    to_return["Name"] = list(set(eggnog_designer_curr["Name"]))[0]
    to_return["Community"] = list(set(eggnog_designer_curr["Community"]))[0]
    to_return.loc[to_return.AssemblerCluster == 0, ["AssemblerCluster","SimDesignerCluster"]] = "none"
    
    return to_return

# compare identified KO pathways combined with cluster info.
def compare_designer_clusts(eggnog_designer_curr, 
                            cluster_file,
                            final_res, functional="KEGG_Pathway"):
    eggnog_designer_curr[functional] = [curr.split(",") if\
                                        type(curr) != list \
                                        else curr for curr in eggnog_designer_curr[functional]]
    eggnog_designer_curr = eggnog_designer_curr.explode(functional)
    eggnog_designer_fct = eggnog_designer_curr.rename({"query":"DesignerQuery",
                                                       functional:"Designer_"+functional},axis="columns")
    cluster_file["MemberSplit"] = [curr.split("_/")[0] for curr in cluster_file["Member"]]
    eggnog_designer_fct = eggnog_designer_fct.merge(cluster_file, left_on = "DesignerQuery",
                                                    right_on = "MemberSplit", how = "left")
    
    final_res = final_res.rename({"query":"CAGQuery",functional:"CAG_"+functional},axis="columns")
    #final_res = final_res.merge(cluster_file, left_on = "CAGQuery",
    #                            right_on = "Member", how = "left")
    
    #return final_res
    to_return = eggnog_designer_fct[["Name","Community","Representative",
                                    "DesignerQuery","Designer_"+functional]].merge(final_res[["Name",
                                                                                              "Community",
                                                                                              "Representative",
                                                                                              "CAGQuery",
                                                                                              "CAG_"+functional]],
                                                                                  left_on = ["Representative",
                                                                                             "Name","Community"],
                                          right_on = ["Representative","Name","Community"],
                              how = "inner").fillna(0)
    def create_list(x):
        return sorted(list(set(x)))
    return to_return[["Name","Community","Representative","CAGQuery","DesignerQuery","Designer_"+functional,
                      "CAG_"+functional]].groupby(["Name","Community","Representative"]).\
            agg({"Designer_"+functional: create_list, "CAG_"+functional: create_list})

def compare_designer_clusts_eukulele(EUKulele_designer_curr, 
                            cluster_file,
                            final_res, tax_level = "Genus"):
    EUKulele_designer_curr = EUKulele_designer_curr.rename({tax_level:"Designer_" + tax_level},
                                                           axis="columns")
    cluster_file["MemberSplit"] = [curr.split("_/")[0].split(".p")[0] for curr in cluster_file["Member"]]
    EUKulele_designer_curr = EUKulele_designer_curr.merge(cluster_file, left_on = "transcript_name",
                                                          right_on = "MemberSplit", how = "inner")
    
    final_res = final_res.rename({tax_level:"CAG_"+tax_level},axis="columns")
    final_res = final_res.merge(cluster_file, left_on = "transcript_name",
                                right_on = "MemberSplit", how = "inner")
    #return final_res
    to_return = EUKulele_designer_curr[["Representative","Name",
                                        "Community","Designer_"+tax_level]].merge(final_res[["Representative",
                                                                                             "Name",
                                                                                             "Community",
                                                                                             "CAG_"+tax_level]],
                                                       left_on = ["Representative","Name","Community"],
                                          right_on = ["Representative","Name","Community"],
                              how = "inner").fillna(0)
    def create_list(x):
        result = sorted(list(set(x)))
        if (len(result) == 1):
            return result[0].strip()
    return to_return[["Name","Community","Representative","Designer_"+tax_level,
                      "CAG_"+tax_level]].groupby(["Name","Community","Representative"]).\
            agg({"Designer_"+tax_level: create_list, "CAG_"+tax_level: create_list})


In [6]:
## READ IN EUKRHYTHMIC DATA##
jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]
all_results = pd.DataFrame()
count_cluster_all = pd.DataFrame()
all_go_comparison = pd.DataFrame()
all_compared_designer = pd.DataFrame()
all_eukulele_compare = pd.DataFrame()
all_EUKulele_CAG = pd.DataFrame()
all_eukulele_designer = pd.DataFrame()
all_clust_designer = pd.DataFrame()
all_eggnog_designer = pd.DataFrame()
all_eggnog_CAG = pd.DataFrame()
eggnog_all = pd.DataFrame()

eggnog_names = ["query","seed_ortholog","evalue","score","eggNOG_OGs","max_annot_lvl","COG_category",
                "Description","Preferred_name","GOs","EC","KEGG_ko","KEGG_Pathway","KEGG_Module",
                "KEGG_Reaction","KEGG_rclass","BRITE","KEGG_TC","CAZy","BiGG_Reaction","PFAMs"]

## SAVE DATA ON CLUSTERING, EGGNOG & EUKULELE, AND CLUSTER METADATA ##
for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        sourmash_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                 "sourmash_MMETSP", "all.csv"))
        community_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                  "03-community_spec", "communities.csv"))
        dictionary_protein = dict()
        for ind_num in range(len(community_file.index)):
            nucl_ids = [r.id for r in SeqIO.parse(community_file.Organism[ind_num], "fasta")]
            prot_ids = [r.id for r in SeqIO.parse(community_file.Protein_Files[ind_num], "fasta")]
            dictionary_protein.update(dict(zip(prot_ids,nucl_ids)))
        
        final_scores = compute_sourmash_index(sourmash_file, community_file)
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        print(jeukebox_dir,"passed prot clust",flush=True)
        cluster_results = pd.DataFrame()
        for sim_file in os.listdir(prot_clust_dir):
            if "tsv" not in sim_file:
                continue
                
            ## WHAT'S INSIDE THE CAG FINAL CONTIGS??
            cluster_result = pd.read_csv(os.path.join(prot_clust_dir,
                                                      sim_file),sep="\t",
                                        header=None,names=["Representative","Member"])
            comm_num = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Name"] = jeukebox_dir
            cluster_result["Assembler_Rep"] = [curr.split("_")[0] for curr in cluster_result.Representative]
            cluster_result["Assembler_Mem"] = [curr.split("_")[0] for curr in cluster_result.Member]
            cluster_results = pd.concat([cluster_results, cluster_result])
            
            
            ## EGGNOG RESULTS
            eggnog_designer_curr = read_eggnog_results(base_dir, jeukebox_dir,
                                                       eggnog_dir="eggnog_designer",
                                                       eggnog_prefix="designer_assembly_",
                                                       sim_file=sim_file,
                                                       eggnog_names=eggnog_names,
                                                       is_prot = True,
                                                       prot_dict = dictionary_protein,
                                                       salmon_dir = os.path.join(base_dir,jeukebox_dir,
                                                                                    "08-salmon_mapping"))
            
            eggnog_all = pd.concat([eggnog_all,read_eggnog_annots(jeukebox_dir, comm_num)])

            all_eggnog_designer = pd.concat([all_eggnog_designer,eggnog_designer_curr])
            eggnog_CAG_curr = read_eggnog_results(base_dir, jeukebox_dir,
                                                   eggnog_dir="eggnog_CAG",
                                                   eggnog_prefix="sim_raw_reads_",
                                                   sim_file=sim_file,
                                                   eggnog_names=eggnog_names,
                                                   salmon_dir = os.path.join(base_dir,
                                                                             jeukebox_dir,
                                                                             "eukrhythmic_assembly",
                                                                             "intermediate-files",
                                                                             "04-compare",
                                                                             "09-CAG-mapping",
                                                                             "salmon"))
            all_eggnog_CAG = pd.concat([all_eggnog_CAG,eggnog_CAG_curr])
            
            # this DataFrame contains the name of every cluster member and the assemblers
            # inside of the cluster.
            CAG_contig_correspondence = convert_names(eggnog_CAG_curr, cluster_result)
            
            ## EUKULELE RESULTS
            EUKulele_CAG = read_EUKulele_annotations(base_dir, jeukebox_dir, "EUKulele_CAG_newnames",
                                                     sim_file)
            all_EUKulele_CAG = pd.concat([all_EUKulele_CAG,EUKulele_CAG])
            
            
            EUKulele_designer = read_EUKulele_annotations(base_dir, jeukebox_dir, "EUKulele_designer", sim_file,
                                                          filename_stub = "designer_assembly_", suffix = "",
                                                          salmon_dir = os.path.join(base_dir,jeukebox_dir,
                                                                                    "08-salmon_mapping"),
                                                          salmon_stub = "")
            
            EUKulele_designer["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            EUKulele_designer["Name"] = jeukebox_dir
            all_eukulele_designer = pd.concat([all_eukulele_designer,EUKulele_designer])
            
            EUKulele_designer_noquant = read_EUKulele_annotations(base_dir, jeukebox_dir, 
                                                                  "EUKulele_designer", sim_file,
                                                          filename_stub = "designer_assembly_", suffix = ".pep",
                                                          salmon_dir = None)
            
            ## CLUSTERING BETWEEN SIM AND DESIGNER
            cluster_file = os.path.join(base_dir, jeukebox_dir, 
                                                      "clustering",
                                                      "clustering_sim_designer",
                                                      "prot-merge","c_0.95","seqid_0.95",
                                                      "sim_raw_reads_" + \
                                                      str(comm_num) + \
                                                      "_and_designer_assembly_" + \
                                                      str(comm_num) + ".tsv")
            if os.path.isfile(cluster_file):
                clust_designer = cluster_sim_designer(cluster_file)
                clust_designer_ret = clust_designer.copy()
                clust_designer_ret["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
                clust_designer_ret["Name"] = jeukebox_dir
                all_clust_designer = pd.concat([all_clust_designer,clust_designer_ret])
                if (eggnog_CAG_curr is not None) & (eggnog_designer_curr is not None):
                    
                    intermediate_cag = eggnog_CAG_curr.merge(clust_designer, left_on = "query",
                                                             right_on = "id_shortened", how = "left")
                    intermediate_cag["id_shortened"] = [curr.split(".p")[0] for curr in intermediate_cag["query"]]

                    final_res = intermediate_cag.merge(CAG_contig_correspondence,
                                                       left_on = ["Community","Name","id_shortened"],
                                                       right_on = ["Community","Name","Representative"], how = "left")\
                                .rename({"GroupType_x":"SimDesignerCluster",
                                         "GroupType_y": "AssemblerCluster"}, axis='columns')

                    count_cluster_type = final_res.groupby(["SimDesignerCluster","AssemblerCluster",
                                                            "Name","Community"])["query"].\
                                                   count().reset_index().pivot(index = "AssemblerCluster",
                                                                               columns = "SimDesignerCluster",
                                                                               values = "query").reset_index()
                    count_cluster_all = pd.concat([count_cluster_all, count_cluster_type])


                    final_res["GOs"] = [curr.split(",") for curr in final_res.GOs]
                    final_res["KEGG_Pathway"] = [curr.split(",") for curr in final_res.KEGG_Pathway]
                    final_res_GOs = final_res.explode("GOs")
                    final_res_KEGG_paths = final_res.explode("KEGG_Pathway")

                    ## COMPARE GO TERMS TO DESIGNER GO TERMS
                    go_comparison = compare_to_designer_go(eggnog_designer_curr, final_res_GOs)
                    all_go_comparison = pd.concat([all_go_comparison,go_comparison])

                    intermediate_cag["KEGG_Pathway"] = [curr.split(",") for curr in intermediate_cag.KEGG_Pathway]
                    intermediate_cag_KEGGs = intermediate_cag.explode("KEGG_Pathway")

                    compared_designer = compare_designer_clusts(eggnog_designer_curr,
                                                                clust_designer,
                                                                intermediate_cag_KEGGs,
                                                                functional="KEGG_Pathway")
                    all_compared_designer = pd.concat([all_compared_designer,compared_designer])
                
            if (EUKulele_designer_noquant is not None) & (EUKulele_CAG is not None) &\
               (not EUKulele_designer_noquant.empty):
                EUKulele_compare = compare_designer_clusts_eukulele(EUKulele_designer_noquant, 
                                                                    clust_designer,
                                                                    EUKulele_CAG,
                                                                    tax_level="Genus").reset_index()
                all_eukulele_compare = pd.concat([all_eukulele_compare,EUKulele_compare])
        
        cluster_results = cluster_results.merge(final_scores,left_on="Community",right_on="Community",how="left")
        
        all_results = pd.concat([all_results,cluster_results])

jEUKebox-Trial3-CommB passed prot clust
jEUKebox-Trial2-CommB passed prot clust
jEUKebox-Trial4-100k passed prot clust
jEUKebox-Trial2-100k passed prot clust
jEUKebox-Trial1-CommB passed prot clust
jEUKebox-Trial3-100k passed prot clust
jEUKebox-Trial1-100k passed prot clust
jEUKebox-Trial4-CommB passed prot clust


In [19]:
## get protein EUKulele annotations for designer
eukulele_prot_annots = pd.DataFrame()
eggnog_prot_annots = pd.DataFrame()
for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        #print(jeukebox_dir,"passed sourmash",flush=True)
        sourmash_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                 "sourmash_MMETSP", "all.csv"))
        community_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                  "03-community_spec", "communities.csv"))
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        print(jeukebox_dir,"passed prot clust",flush=True)
        cluster_results = pd.DataFrame()
        for sim_file in os.listdir(prot_clust_dir):
            #eukulele_prot_annots
            
            EUKulele_designer = read_EUKulele_annotations(base_dir, jeukebox_dir, "EUKulele_designer", sim_file,
                                                          filename_stub = "designer_assembly_", suffix = ".pep",
                                                          salmon_dir = None)
            
            eggnog_designer_curr = read_eggnog_results(base_dir, jeukebox_dir,
                                                       eggnog_dir="eggnog_designer",
                                                       eggnog_prefix="designer_assembly_",
                                                       sim_file=sim_file,
                                                       eggnog_names=eggnog_names,
                                                       read_salmon=False,
                                                       salmon_dir = os.path.join(base_dir,jeukebox_dir,
                                                                                    "08-salmon_mapping"))
            
            eggnog_designer_curr["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            eggnog_designer_curr["Name"] = jeukebox_dir
            
            eggnog_prot_annots = pd.concat([eggnog_prot_annots,eggnog_designer_curr])
            
            EUKulele_designer["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            EUKulele_designer["Name"] = jeukebox_dir
            eukulele_prot_annots = pd.concat([eukulele_prot_annots,EUKulele_designer])

jEUKebox-Trial3-CommB passed prot clust
jEUKebox-Trial2-CommB passed prot clust
jEUKebox-Trial4-100k passed prot clust
jEUKebox-Trial2-100k passed prot clust
jEUKebox-Trial1-CommB passed prot clust
jEUKebox-Trial3-100k passed prot clust
jEUKebox-Trial1-100k passed prot clust
jEUKebox-Trial4-CommB passed prot clust


In [20]:
designer_species = dict(zip(eukulele_prot_annots.transcript_name, eukulele_prot_annots.Species))
designer_genera = dict(zip(eukulele_prot_annots.transcript_name, eukulele_prot_annots.Genus))
designer_kos = dict(zip(eggnog_prot_annots["query"],eggnog_prot_annots.KEGG_ko))

In [21]:

jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]

cag_sizes=pd.DataFrame()
designer_sizes=pd.DataFrame()

for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    #if jeukebox_dir == "jEUKebox-Trial2-CommB":
    #    continue
    #print(jeukebox_dir,"starting")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        #print(jeukebox_dir,"passed sourmash",flush=True)
        sourmash_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                 "sourmash_MMETSP", "all.csv"))
        community_file = pd.read_csv(os.path.join(base_dir, jeukebox_dir, 
                                                  "03-community_spec", "communities.csv"))

        #print(final_scores,flush=True)
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        for sim_file in os.listdir(prot_clust_dir):
            if "fasta" not in sim_file:
                continue
                
                
            comm_num = sim_file.split("sim_raw_reads_")[-1].split("_")[0]
                
            ## STORED CONTIGS FROM DESIGNER ASSEMBLY
            designer_contigs = os.path.join(base_dir, jeukebox_dir, 
                                                      "06-designer_assemblies",
                                                      "designer_assembly_" + str(comm_num) + ".fasta")
            designer_size = os.path.getsize(designer_contigs)
            des_length_distro = [len(r.seq) for r in SeqIO.parse(designer_contigs, "fasta")]
            des_gc_content = [r.seq.count("G") + r.seq.count("C") for r in SeqIO.parse(designer_contigs, "fasta")]
            des_ids = [r.id for r in SeqIO.parse(designer_contigs, "fasta")]
            
            ## WHAT'S INSIDE THE CAG ASSEMBLY
            cag_assembly = os.path.join(prot_clust_dir,sim_file)
            cag_length_distro = [len(r.seq) for r in SeqIO.parse(cag_assembly, "fasta")]
            cag_gc_content = [r.seq.count("G") + r.seq.count("C") for r in SeqIO.parse(cag_assembly, "fasta")]
            cag_ids = [r.id for r in SeqIO.parse(cag_assembly, "fasta")]
            cag_size = os.path.getsize(cag_assembly)
            
            mmetsp_group = "B"
            if "100k" in jeukebox_dir:
                mmetsp_group = "A"
            
            curr_df = pd.DataFrame({"jEUKebox": [jeukebox_dir] * len(cag_length_distro),
                                    "Representative": cag_ids,
                                    "Sizes": cag_length_distro,
                                    "GC": cag_gc_content,
                                    "Community": comm_num,
                                    "MMETSPGroup": mmetsp_group})
            cag_sizes = pd.concat([cag_sizes,curr_df])
            
            
            curr_df = pd.DataFrame({"jEUKebox": [jeukebox_dir] * len(des_length_distro),
                                    "Representative": des_ids,
                                    "Sizes": des_length_distro,
                                    "GC": des_gc_content,
                                    "Community": comm_num,
                                    "MMETSPGroup": mmetsp_group})
            designer_sizes = pd.concat([designer_sizes,curr_df])

In [22]:
cag_sizes["Community"] = [int(curr) for curr in cag_sizes["Community"]]

designer_sizes["Source"] = "Designer"
cag_sizes["Source"] = "Reassembled"

concat_sizes = pd.concat([designer_sizes,cag_sizes])
concat_sizes["GCFract"] = concat_sizes["GC"] / concat_sizes["Sizes"]
compared_sizes = concat_sizes.groupby(["MMETSPGroup","Community","Source"]).describe().reset_index()

compared_sizes.columns = [' '.join(col).strip() for col in compared_sizes.columns.values]

In [23]:
compared_species = pd.merge(all_eukulele_designer.groupby(["Genus","Name","Community"])["TPM"].\
                             sum().reset_index().\
         rename(columns={"TPM":"DesignerGenusTPM"}),
         all_EUKulele_CAG.groupby(["Genus","Name","Community"])["TPM"].sum().reset_index().\
         rename(columns={"TPM":"CAGGenusTPM"}),
         left_on=["Name","Community","Genus"],right_on=["Name","Community","Genus"])
compared_species["MMETSPGroup"] = ["A" if "100k" in curr else "B" for curr in compared_species["Name"]]
compared_species["Genus"] = [curr.strip() for curr in compared_species["Genus"]]

In [35]:
## GET THE FILE SIZES TO GENERATE TABLES FOR PAPER ##
communities = list(range(1,7))
file_sizes = pd.DataFrame()
jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]
for jeukebox_dir in jeukebox_dirs:
    print(jeukebox_dir,flush=True)
    for comm_num in communities:
        base_dir_covs = os.path.join(base_dir, jeukebox_dir, "clustering",
                                    "clustering_designer","prot")
        coverage_dirs = os.listdir(base_dir_covs)
        all_seq_files = pd.DataFrame()
        for coverage in coverage_dirs:
            cov_curr = coverage.split("_")[-1]
            seq_ids = os.listdir(os.path.join(base_dir_covs,coverage))
            for seq_id in seq_ids:
                seq_curr = seq_id.split("_")[-1]
                seq_file = os.path.join(base_dir_covs,coverage,
                                                                 seq_id,
                                                                 "designer_assembly_" + \
                                    str(comm_num) + ".pep.fasta")
                contig_no = len([r.id for r in SeqIO.parse(seq_file, "fasta")])
                kegg_kos = []
                [kegg_kos.extend(designer_kos[curr.id].split(",")) if curr.id in\
                    designer_kos else "" for curr in SeqIO.parse(seq_file, "fasta")]
                kegg_kos = set(kegg_kos)
                genera = set([designer_genera[curr.id].strip() if curr.id in designer_genera \
                               else "" for curr in SeqIO.parse(seq_file, "fasta")])
                genera = list(genera)
                genera.remove("")
                genera.remove("Unannotated")
                species = set([designer_species[curr.id].strip() if curr.id in designer_species \
                               else "" for curr in SeqIO.parse(seq_file, "fasta")])
                species = list(species)
                species.remove("")
                species.remove("Unannotated")
                file_size = os.path.getsize(seq_file)
            
                mmetsp_group = "B"
                if "100k" in jeukebox_dir:
                    mmetsp_group = "A"

                curr_df = pd.DataFrame({"jEUKebox": [jeukebox_dir],
                                        "DesignerSize": file_size,
                                        "DesignerContigs": contig_no,
                                        "SeqID": seq_id,
                                        "Coverage": coverage,
                                        "Community": comm_num,
                                        "MMETSPGroup": mmetsp_group,
                                        "Species": "-".join(sorted(species)),
                                        "NumSpecies": len(species),
                                        "Genera": "-".join(sorted(genera)),
                                        "NumGenera": len(genera),
                                        # "KOs": kegg_kos,
                                        "NumKOs": len(kegg_kos)})
                file_sizes = pd.concat([file_sizes,curr_df])

jEUKebox-Trial3-CommB
jEUKebox-Trial2-CommB
jEUKebox-Trial4-100k
jEUKebox-Trial2-100k
jEUKebox-Trial1-CommB
jEUKebox-Trial3-100k
jEUKebox-Trial1-100k
jEUKebox-Trial4-CommB


In [46]:
### Read in BLAST results on CAG ###

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))
output_dir="two_way_contigs" #four_way_contigs
num_in_common = 2
os.system("mkdir -p " + output_dir)
## READ IN EUKRHYTHMIC DATA##
jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]
all_with_blast_eval_10_2 = pd.DataFrame()

def create_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    return "-".join(sorted(list(set(y))))

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))

for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    #if jeukebox_dir == "jEUKebox-Trial2-CommB":
    #    continue
    #print(jeukebox_dir,"starting")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        prot_proteins_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "04-compare", "08-CAG-proteins")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        print(jeukebox_dir,"passed prot clust",flush=True)
        cluster_results = pd.DataFrame()
        for sim_file in os.listdir(prot_clust_dir):
            if "tsv" not in sim_file:
                continue
            fasta_file=sim_file.split(".tsv")[0] + ".fasta"
            ## WHAT'S INSIDE THE CAG FINAL CONTIGS??
            cluster_result = pd.read_csv(os.path.join(prot_clust_dir,
                                                      sim_file),sep="\t",
                                        header=None,names=["Representative","Member"])
            comm_num = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Name"] = jeukebox_dir
            cluster_result["Assembler_Rep"] = [curr.split("_")[0] for curr in cluster_result.Representative]
            cluster_result["Assembler_Mem"] = [curr.split("_")[0] for curr in cluster_result.Member]
            
            four_way_contigs = cluster_result.groupby(["Community","Name","Representative",
                                                       "Assembler_Rep"]).agg({"Assembler_Mem": [count_set,create_set]}).reset_index()
            four_way_contigs.columns = ['_'.join(col).strip() for col in four_way_contigs.columns.values]
            blast_file = pd.read_csv(os.path.join("/vortexfs1/omics/alexander/akrinos/"+\
                                                  "2021-testing-eukrhythmic/eukrhythmic-eLife/"+\
                                                  "snakemake-workflows/blast-snake/eval_10_2_blast_top_1",jeukebox_dir,
                                                  "rhythmic_sim_raw_reads_" + \
                                                  str(comm_num) + "_euk_designer_assembly_" + str(comm_num),
                                                 "blast_formatted.reduced.out"),
                                     sep = "\t") #, header = None,
                                     #names=["qseqid","sseqid","pident","length","mismatch",
                                     #       "gapopen","qstart","qend","sstart","send",
                                     #       "evalue","bitscore"])
            list_ids = [curr.id.split(".")[0].split("sim_raw_reads_"+str(comm_num)+"_")[-1] for curr in \
                        SeqIO.parse(os.path.join(prot_proteins_clust_dir,
                                                 "sim_raw_reads_" + str(comm_num) + "_CAG.fasta.transdecoder.pep"),"fasta")]
            protein_df = pd.DataFrame({"Protein": list_ids, "BlastMatch": "No"})
            
            
            blast_file["RepresentativeFix"] = [str(curr).split(".")[0].split("sim_raw_reads_"+str(comm_num)+"_")[-1] for \
                                    curr in blast_file["qseqid"]]
            
            blast_file = pd.merge(blast_file,protein_df,how="right",left_on="RepresentativeFix",right_on="Protein")
            blast_file.loc[blast_file.qseqid == blast_file.qseqid,"BlastMatch"] = "Yes"
            blast_file["qseqid"] = [str(curr).split(".")[0].split("sim_raw_reads_"+str(comm_num)+"_")[-1] for \
                                    curr in blast_file["qseqid"]]
            four_way_contigs["RepresentativeFix"] = [str(curr).split(".")[0].split("sim_raw_reads_"+str(comm_num)+"_")[-1] for \
                                    curr in four_way_contigs["Representative_"]]
            combined_blast = pd.merge(blast_file,four_way_contigs,right_on="RepresentativeFix",left_on="Protein",
                                      how = "outer")
            combined_blast.loc[combined_blast.BlastMatch != combined_blast.BlastMatch,"BlastMatch"] = "NoProtein"
            combined_blast.loc[combined_blast.BlastMatch != combined_blast.BlastMatch,"pident"] = -1
            all_with_blast_eval_10_2 = pd.concat([all_with_blast_eval_10_2,combined_blast])

jEUKebox-Trial3-CommB passed prot clust
jEUKebox-Trial2-CommB passed prot clust
jEUKebox-Trial4-100k passed prot clust
jEUKebox-Trial2-100k passed prot clust
jEUKebox-Trial1-CommB passed prot clust
jEUKebox-Trial3-100k passed prot clust
jEUKebox-Trial1-100k passed prot clust
jEUKebox-Trial4-CommB passed prot clust


In [47]:
all_with_blast_filled = all_with_blast_eval_10_2.fillna({'pident':0})
all_with_blast_filled = all_with_blast_filled[all_with_blast_filled.pident != "pident"]
all_with_blast_filled["num_pident"] = all_with_blast_filled["pident"].astype(float)

all_with_blast_filled["pident_cat"] = ["Missing" if num_curr == 0 else "Above75" if num_curr > 75 else "Below75" if num_curr > 50 else "Below50" for num_curr in all_with_blast_filled.num_pident]
num_pident_track = all_with_blast_filled.groupby(["Assembler_Mem_count_set","Name_","pident_cat"])["num_pident"].count().reset_index()
all_with_blast_filled.groupby(["Assembler_Mem_create_set","pident_cat","Name_"])["pident"].count().reset_index()

,Assembler_Mem_create_set,pident_cat,Name_,pident
0,megahit,Above75,jEUKebox-Trial1-100k,15488
1,megahit,Above75,jEUKebox-Trial1-CommB,13790
2,megahit,Above75,jEUKebox-Trial2-100k,19222
3,megahit,Above75,jEUKebox-Trial2-CommB,9740
4,megahit,Above75,jEUKebox-Trial3-100k,16936
...,...,...,...,...
475,trinity,Missing,jEUKebox-Trial2-CommB,1777
476,trinity,Missing,jEUKebox-Trial3-100k,2863
477,trinity,Missing,jEUKebox-Trial3-CommB,2454
478,trinity,Missing,jEUKebox-Trial4-100k,2162


In [49]:
all_with_blast_filled.groupby(["Assembler_Mem_create_set","pident_cat","Name_","Community_"])["pident"].count().reset_index().\
    groupby(["Assembler_Mem_create_set","pident_cat"])["pident"].agg(["mean","std"])\
    .reset_index().to_csv(os.path.join("..","data-output", "all_with_blast_filled." + str(date.today()) + ".csv"))

In [36]:
check_true_ans = pd.merge(compared_species,file_sizes[["Community","MMETSPGroup","Genera","jEUKebox"]],
         left_on = ["Community","MMETSPGroup","Name"],
         right_on = ["Community","MMETSPGroup","jEUKebox"])

check_true_ans["Correct"] = ["Unannotated" if (gen == "Unannotated") | (gen == "-") | (gen == "Uncertain") else \
                             "Match" if (gen in genera) else "Conflict" for gen,genera in zip(check_true_ans.Genus,
                                                                                      check_true_ans.Genera)]

def create_list_set(x):
    return "-".join(sorted(list(set(x))))

def measure_list_set(x):
    return len(list(set(x)))

count_genera = pd.melt(check_true_ans, value_vars = ["DesignerGenusTPM","CAGGenusTPM"],
        id_vars=["Community","MMETSPGroup","Correct","Name","Genus"],value_name="TPM",var_name="Source").\
        groupby(["Community","MMETSPGroup","Correct","Source","Name","Genus"])["TPM"].sum().reset_index()

count_genera = count_genera[count_genera.TPM != 0]
count_genera = count_genera.groupby(["Community","MMETSPGroup","Correct","Name","Source"]).\
    agg({"Genus":[create_list_set,measure_list_set]}).reset_index()

count_genera.columns = [' '.join(col).strip() for col in count_genera.columns.values]

In [38]:
correct_uncorrect = pd.melt(check_true_ans, value_vars = ["DesignerGenusTPM","CAGGenusTPM"],
        id_vars=["Community","MMETSPGroup","Correct","Name"],value_name="TPM",var_name="Source").\
        groupby(["Community","MMETSPGroup","Correct","Source","Name"])["TPM"].sum().reset_index()
correct_uncorrect["Source"] = ["reassembly" if "CAG" in curr else "designer" for curr in correct_uncorrect.Source]

In [26]:
kegg_ko_incidence = eggnog_all.assign(KEGG_ko=eggnog_all.KEGG_ko.str.split(",")).explode('KEGG_ko').\
    groupby(["KEGG_ko","Source","jEUKebox","MMETSPGroup"])["query"].count().reset_index()

In [27]:
go_term_incidence = eggnog_all.assign(GOs=eggnog_all.GOs.str.split(",")).explode('GOs').\
    groupby(["GOs","Source","jEUKebox","MMETSPGroup"])["query"].count().reset_index()

In [28]:
kegg_ko_incidence_plot = kegg_ko_incidence.pivot_table(index = ["KEGG_ko","jEUKebox","MMETSPGroup"],
                                                 columns = "Source", 
                                                 values = "query").fillna(0).reset_index()

In [44]:
number_cats = all_results.groupby(["Name","Community","Representative"])["Assembler_Mem"].\
    apply(lambda x: "-".join(sorted(list(set(x))))).reset_index().groupby(["Name","Community","Assembler_Mem"]).count().\
    reset_index()

In [2]:
## WRITE OUT RESULTS FOR USE ELSEWHERE ##
all_results.to_csv(os.path.join("..","data-output", "all_results." + str(date.today()) + ".csv"))
all_eukulele_compare.to_csv(os.path.join("..","data-output", "all_eukulele_compare." + str(date.today()) + ".csv"))
all_eukulele_designer.to_csv(os.path.join("..","data-output", "all_eukulele_designer." + str(date.today()) + ".csv"))
all_compared_designer.to_csv(os.path.join("..","data-output", "all_compared_designer." + str(date.today()) + ".csv"))
compared_species.to_csv(os.path.join("..","data-output", "compared_species." + str(date.today()) + ".csv"))
count_cluster_all.to_csv(os.path.join("..","data-output", "count_cluster_all." + str(date.today()) + ".csv"))
eukulele_prot_annots.to_csv(os.path.join("..","data-output", "eukulele_prot_annots." + str(date.today()) + ".csv"))
all_eggnog_CAG.to_csv(os.path.join("..","data-output", "all_eggnog_CAG." + str(date.today()) + ".csv"))

NameError: name 'all_results' is not defined

In [8]:
all_eggnog_CAG.to_csv(os.path.join("..","data-output", "all_eggnog_CAG." + str(date.today()) + ".csv"))
all_eggnog_designer.to_csv(os.path.join("..","data-output", "all_eggnog_designer." + str(date.today()) + ".csv"))

In [10]:
all_eukulele_designer.to_csv(os.path.join("..","data-output", "all_eukulele_designer." + str(date.today()) + ".csv"))
all_EUKulele_CAG.to_csv(os.path.join("..","data-output", "all_EUKulele_CAG." + str(date.today()) + ".csv"))

In [30]:
cag_sizes.to_csv(os.path.join("..","data-output", 
                                    "cag_sizes." + str(date.today()) + ".csv"))
designer_sizes.to_csv(os.path.join("..","data-output", 
                                    "designer_sizes." + str(date.today()) + ".csv"))

compared_sizes.to_csv(os.path.join("..","data-output", 
                                    "compared_sizes." + str(date.today()) + ".csv"))

concat_sizes.to_csv(os.path.join("..","data-output", 
                                    "concat_sizes." + str(date.today()) + ".csv"))
kegg_ko_incidence.to_csv(os.path.join("..","data-output", 
                                    "kegg_ko_incidence." + str(date.today()) + ".csv"))
count_genera.to_csv(os.path.join("..","data-output", 
                                    "count_genera." + str(date.today()) + ".csv"))
correct_uncorrect.to_csv(os.path.join("..","data-output", 
                                    "correct_uncorrect." + str(date.today()) + ".csv"))
compared_species.to_csv(os.path.join("..","data-output", 
                                    "compared_species." + str(date.today()) + ".csv"))
eggnog_prot_annots.to_csv(os.path.join("..","data-output", 
                                    "eggnog_prot_annots." + str(date.today()) + ".csv"))

In [45]:
number_cats.to_csv(os.path.join("..","data-output", 
                                    "number_cats." + str(date.today()) + ".csv"))

In [37]:
count_genera.to_csv(os.path.join("..","data-output", 
                                    "count_genera." + str(date.today()) + ".csv"))

In [39]:
correct_uncorrect.to_csv(os.path.join("..","data-output", 
                                    "correct_uncorrect." + str(date.today()) + ".csv"))

In [40]:
compared_species.to_csv(os.path.join("..","data-output", 
                                    "compared_species." + str(date.today()) + ".csv"))

In [41]:
eggnog_prot_annots.to_csv(os.path.join("..","data-output", 
                                    "eggnog_prot_annots." + str(date.today()) + ".csv"))

In [31]:
## READ BACK IN RESULTS ##
curr_date = "2022-01-24"#date.today()
all_results = pd.read_csv(os.path.join("..","data-output", "all_results." + str(curr_date) + ".csv"))
all_eukulele_compare = pd.read_csv(os.path.join("..","data-output", "all_eukulele_compare." + str(curr_date) + ".csv"))
all_compared_designer = pd.read_csv(os.path.join("..","data-output", "all_compared_designer." + str(curr_date) + ".csv"))
count_cluster_all = pd.read_csv(os.path.join("..","data-output", "count_cluster_all." + str(curr_date) + ".csv"))
eukulele_prot_annots = pd.read_csv(os.path.join("..","data-output", "eukulele_prot_annots." + str(curr_date) + ".csv"))
cag_sizes=pd.read_csv(os.path.join("..","data-output", 
                                    "cag_sizes." + str(curr_date) + ".csv"))